In [ ]:
!pip install git+https://github.com/suno-ai/bark.git && \
  pip uninstall -y torch torchvision torchaudio && \
  pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118

!pip install flask flask-ngrok
!pip install flask-cors
!pip install pyngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install --upgrade flask
import subprocess

subprocess.Popen(['./ngrok', 'http', '5000'])

from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio

preload_models()

# Get the API link from this cell
Note: if the link doesn't start with 'https' and start with only 'http' Restart the  cell or it will not work! it might take a couple restarts to get a link with https

In [ ]:

get_ipython().system_raw('./ngrok http 5000 &')
!sleep 1
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("if this ^ doesn't start with 'https' and start with only 'http' Restart the this cell or it will not work!")
print("Also this is the link that you need for the extention")
from scipy.io.wavfile import write

from flask import Flask, request, make_response
import numpy as np
from flask import Flask, request, jsonify
from bark import SAMPLE_RATE, generate_audio, preload_models
import base64
import io
from flask_cors import CORS
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
from scipy.io.wavfile import write
import io
import base64
from bark.generation import SUPPORTED_LANGS
AVAILABLE_PROMPTS = ["Unconditional", "Announcer"]
PROMPT_LOOKUP = {}
DEBUG_MODE = False
for _, lang in SUPPORTED_LANGS:
    for n in range(10):
        label = f"Speaker {n} ({lang})"
        AVAILABLE_PROMPTS.append(label)
        PROMPT_LOOKUP[label] = f"{lang}_speaker_{n}"
PROMPT_LOOKUP["Unconditional"] = None
PROMPT_LOOKUP["Announcer"] = "announcer"

app = Flask(__name__)
CORS(app)
def gen_tts(text, history_prompt):#, temp_semantic, temp_waveform):
    history_prompt = PROMPT_LOOKUP[history_prompt]
    if DEBUG_MODE:
        audio_arr = np.zeros(SAMPLE_RATE)
    else:
        audio_arr = generate_audio(text, history_prompt=history_prompt)#, text_temp=temp_semantic, waveform_temp=temp_waveform)
    audio_arr = (audio_arr * 32767).astype(np.int16)
    return (SAMPLE_RATE, audio_arr)
@app.route('/synthesize', methods=['POST'])
def synthesize():
    text = request.form.get('text')

    print("text:",text)
    history_prompt = request.form.get('history_prompt')
    print("history_prompt:",history_prompt)
    sample_rate, audio_arr = gen_tts(text, history_prompt)

    audio_bytes = io.BytesIO()
    write(audio_bytes, sample_rate, audio_arr)
    audio_bytes.seek(0)
    audio_base64 = base64.b64encode(audio_bytes.read()).decode('utf-8')

    return jsonify({'audio_data': audio_base64})


if __name__ == '__main__':
    preload_models()
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)